# RBC

In [1]:
import requests
import numpy as np
import datetime
import re
from bs4 import BeautifulSoup

from fake_useragent import UserAgent

import time
import datetime
import pandas as pd
import pickle
import time

from tqdm import tqdm
from joblib import Parallel, delayed

# 1. Download hrefs

In [4]:
news_titles = [ ]
fail_cnt = 0
bla_cnt = 0

In [5]:
for offset in range(0, 100000, 100):
    time.sleep(1)
    url = 'https://www.rbc.ru/filter/ajax?offset={}&limit=100'.format(offset)
    response = requests.get(url)
    html = response.json()['html']
    soup = BeautifulSoup(html,'html.parser')
    a = soup.find_all('div')
    
    diff = response.json()['count'] - len(a)
    fail_cnt += diff
    
    print(offset)
    
    for item in a:
        if item['class'][0] in {'item__image-wrap'}:
            print('fuck')
            bla_cnt += 1
            continue 
            
        cur = { }
        cur['href'] = item.find('a').get('href')
        if item['class'][0] == 'item':
            cur['title'] = item.find('span', {'class' : 'item__title'}).text.strip()
            cur['time'] = item.find('span', {'class' : 'item__info'}).text.strip()
            try:
                cur['snippet'] = item.find('span', {'class' : 'item__text'}).text.strip()
            except:
                cur['snippet'] = None   
        else:
            cur['title'] = item.find('span', {'class' : 'item-opinion__title'}).text.strip()
            cur['time'] = item.find('span', {'class' : 'item-opinion__info'}).text.strip()
            
            try:
                cur['snippet'] = item.find('span', {'class' : 'item-opinion__text'}).text.strip()
            except:
                cur['snippet'] = None               
        
        news_titles.append(cur)   
        
    print(cur['time'])

0
24 окт, 08:31
100
23 окт, 12:05
200
22 окт, 17:37
300
22 окт, 01:35
400
21 окт, 10:33
500
20 окт, 14:37
600
19 окт, 19:45
700
19 окт, 01:00
800
17 окт, 21:36
900
16 окт, 20:41
1000
16 окт, 06:53
1100
15 окт, 12:02
1200
14 окт, 17:01
1300
13 окт, 23:00
1400
13 окт, 10:04
1500
12 окт, 13:30
1600
11 окт, 15:05
1700
10 окт, 12:00
1800
09 окт, 13:37
1900
08 окт, 16:23
2000
07 окт, 19:54
2100
07 окт, 04:17
2200
06 окт, 12:09
2300
05 окт, 16:55
2400
04 окт, 22:17
2500
03 окт, 18:13
2600
бля
02 окт, 17:13
2700
01 окт, 22:44
2800
01 окт, 08:45
2900
30 сен, 13:01
3000
29 сен, 18:32
3100
29 сен, 00:21
3200
28 сен, 10:40
3300
27 сен, 09:17
3400
26 сен, 04:59
3500
25 сен, 11:31
3600
24 сен, 15:52
3700
23 сен, 21:34
3800
23 сен, 10:19
3900
22 сен, 14:47
4000
21 сен, 18:27
4100
21 сен, 00:30
4200
19 сен, 13:33
4300
18 сен, 13:33
4400
17 сен, 16:37
4500
16 сен, 20:33
4600
16 сен, 02:08
4700
15 сен, 10:31
4800
14 сен, 14:32
4900
13 сен, 16:16
5000
12 сен, 13:17
5100
11 сен, 13:53
5200
10 сен, 17:31
5

KeyboardInterrupt: 

In [6]:
len(news_titles), bla_cnt, fail_cnt

(17700, 5, -5)

In [24]:
hrfs = [item['href'] for item in news_titles]
len(hrfs), len(set(hrfs))

(17407, 17405)

In [28]:
# news_titles = news_titles[:-293]
news_titles[-1]['time']

'01 июн, 00:08'

In [60]:
!ls news_data/1.rbc

dirty                      new_titles.pickle
lemm                       rbc_titles_2020_new.pickle


In [33]:
with open('news_data/1.rbc/rbc_titles_2020_new.pickle', 'wb') as f:
      pickle.dump(news_titles, f)

# 2. Download news

In [34]:
with open('news_data/1.rbc/rbc_titles_2020_new.pickle', 'rb') as f:
    news_titles = pickle.load(f)
    
len(news_titles)

17407

In [36]:
def get_rbc_news(item):
    cur = { }
    try:
        url = item['href']
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html,'html.parser')
        
        cur['category'] = soup.find('a', {'class': 'article__header__category'}).text
        cur['date'] = soup.find('span', {'class': 'article__header__date'})['content']
        cur['text'] = soup.find('div', {'class': 'article__text'}).text
    except: 
        print('FUCK!', end='')
    cur['title'] = item['title']
    cur['href'] = item['href']
    return cur
    
def separator(vect, part):
    n = len(vect)
    vec_parts = [round(n/part)*i for i in range(part)]
    vec_parts.append(n)
    out = [vect[vec_parts[i]:vec_parts[i+1]] for i in range(part)]
    return(out)

In [37]:
x_batches = dff_batches = separator(news_titles, 200)
len(x_batches), len(x_batches[0])

(200, 87)

In [38]:
x_new = [ ]
n_jobs = -1

for i,batch in enumerate(x_batches):
    print(f"батч {i}")
    cur_lemm = Parallel(n_jobs=n_jobs)(delayed(get_rbc_news)(
        text) for text in tqdm(batch))
    x_new.extend(cur_lemm)

  0%|          | 0/87 [00:00<?, ?it/s]

батч 0


  0%|          | 0/87 [00:00<?, ?it/s]

батч 1


  0%|          | 0/87 [00:00<?, ?it/s]

...

  0%|          | 0/87 [00:00<?, ?it/s]

батч 198


  0%|          | 0/94 [00:00<?, ?it/s]

батч 199


100%|██████████| 94/94 [00:16<00:00,  5.67it/s]


In [39]:
import pandas as pd
df = pd.DataFrame(x_new)

print(df.shape)

df.drop_duplicates('href', inplace=True)
print(df.shape)
df = df.fillna(' ')
df.head()

(17407, 5)
(17405, 5)


,category,date,text,title,href
0,Бизнес,2020-10-25T13:51:06+03:00,\n\nЛи Гон Хи оставил после себя активов на $1...,Наследники умершего главы Samsung заплатят рек...,https://www.rbc.ru/business/25/10/2020/5f95505...
1,Общество,2020-10-25T13:42:41+03:00,\n\n\n\n\n\n\n ...,СК возбудил дело против стрелявшего в подростк...,https://www.rbc.ru/rbcfreenews/5f9555639a7947a...
2,Пандемия коронавируса,2020-10-25T13:21:57+03:00,\n\n\n\n\n\nОльга Ковитиди\n\n ...,Сенатор от Крыма Ольга Ковитиди заразилась кор...,https://www.rbc.ru/rbcfreenews/5f954da39a7947a...
3,Протесты в Белоруссии,2020-10-25T13:20:12+03:00,\n\n\n\n\n\n\n ...,В Минске перед акцией оппозиции вновь начались...,https://www.rbc.ru/rbcfreenews/5f954a0c9a7947a...
4,Пандемия коронавируса,2020-10-25T13:08:39+03:00,\n\n\n\n\n\nАлександр Мясников\n\n ...,Мясников ответил на требование Жириновского ли...,https://www.rbc.ru/rbcfreenews/5f9548e89a7947a...


In [40]:
df_title = pd.DataFrame(news_titles)
df_title.drop_duplicates('href', inplace=True)
print(df_title.shape)
df_title = df_title.fillna(' ')
df_title.drop('title', axis=1, inplace=True)
df_title.head()

(17405, 4)


,href,time,snippet
0,https://www.rbc.ru/business/25/10/2020/5f95505...,13:51,"Ли Гон Хи оставил после себя активов на $15,6 ..."
1,https://www.rbc.ru/rbcfreenews/5f9555639a7947a...,13:42,
2,https://www.rbc.ru/rbcfreenews/5f954da39a7947a...,13:21,
3,https://www.rbc.ru/rbcfreenews/5f954a0c9a7947a...,13:20,
4,https://www.rbc.ru/rbcfreenews/5f9548e89a7947a...,13:08,


In [41]:
dff = df.set_index('href').join(df_title.set_index('href')).reset_index()
dff.shape

(17405, 7)

In [42]:
rbc_news = dff.to_dict('records')

In [48]:
!ls news_data/1.rbc/dirty

rbcNews2012-2015.pickle   rbcNews2015-2020.pickle   rbcTitles2012-2020.pickle


In [61]:
with open('news_data/1.rbc/dirty/rbcNews2015-2020.pickle', 'rb') as f:
    news_test = pickle.load(f)
    
len(news_test)

216800

In [62]:
len(rbc_news)

17405

In [64]:
news_test.extend(rbc_news)
len(news_test)

234205

In [65]:
with open('news_data/1.rbc/dirty/rbcNews2015-2020.pickle', 'wb') as f:
      pickle.dump(news_test, f)